# Как я делал самую ненужную вещь в мире

Итак, передо мной стояла следующая задача: поменять название беседы класса на $9 + \frac{hour}{2400}$, 
где $hour$ -- число прошедших с начала каникул до текущего момента часов, дождаться следующего момента времени, когда на часах будет сколько-нибудь часов ровно, а затем менять его аналогичным образом каждый час.

Мы собираемся работать с vk в python3. Поэтому из терминала сначала (если ещё не установлены) стоит установить нужные библиотеки:

<code>pip3 install vk</code>
<br />
<code>pip3 install vk_api</code>

Дальнейшее действо происходило в специальной папке D20chat. Там находятся файл chat.py -- в нём лежит код, и файл fin.txt с данными для входа в ВК. fin.txt состоит из одной строки, которая имеет следующий вид:

<code>login password chat_id</code>

(Тут <code>login</code> и <code>password</code> -- мои логин и пароль при входе в ВК, <code>chat_id</code> -- мой chat_id чата d20 в ВК)

Перейду, наконец, к содержанию chat.py. Сперва -- подключение нужных библиотек.

In [18]:
import time
import vk
import vk_api

Сначала учим адекватно извлекать текущие дату и время из строки, которую выдаёт <code>time.ctime()</code>. Посмотрим, собственно на эту строку:

In [20]:
print(time.ctime())

Mon Jun  4 12:00:43 2018


Теперь, кажется, очевидно, как появляются следующие функции:

In [22]:
def date():
    c_time = time.ctime()
    c_time = c_time.split()
    return [c_time[1], c_time[2]]

In [23]:
def cur_time():
    c_time = time.ctime()
    c_time = c_time.split()
    return c_time[3].split(':')

Можно посмотреть, как выглядит вывод:

In [25]:
print(date())

['Jun', '4']


In [26]:
print(cur_time())

['12', '05', '16']


Далее делаем функцию, которая будет считать время до следующего попадания минутной стрелки на 12 в секундах

In [28]:
def to_next_hour(c_time):
    return (60 - int(c_time[1])) * 60 + 60 - int(c_time[2])

Если подставить туда вместо параметра результат функции <code>cur_time()</code>, то получим искомое:

In [30]:
print(to_next_hour(cur_time()))

3057


Чтобы сделать удобнее подсчёт времени, создадим функцию подсчёта дней, прошедших с начала каникул:

In [34]:
def number_of_day(c_time):
    if c_time[0] == 'Jun':
        return int(c_time[1]) + 8 #В мае 2018 года 8 свободных дней
    elif c_time[0] == 'Jul':
        return int(c_time[1]) + 38 #В июне 2018 года 30 свободных дней
    else:
        return int(c_time[1]) + 69 #В июле 2018 года 31 свободный день

Сюда в качестве параметра подставляется <code>date()</code>:

In [36]:
print(number_of_day(date()))

12


Теперь осталось только написать функцию, которая будет менять название беседы:

In [41]:
def old_change_name():
    global u_log, u_pas, uid
    hours = (number_of_day(date()) - 1) * 24 + int(cur_time()[0]) #число часов с начала каникул
    vk = vk_api.VkApi(login = u_log, password = u_pas) 
    #в предыдущей строке глобальные переменные u_log и u_pas, в которых лежат логин и пароль, помогают войти в ВК
    vk.auth() #вход в ВК
    vk.method('messages.editChat', {'chat_id':int(uid), 'title':str(9 + hours / 2400) + ' Д'})
    #в uid лежит строка с chat_id, названием становится необходимая дробь
    return 0

Проверим, какую строку выдаст в название беседы функция, если бы её запустили сегодня, 4 июня, в 11:00 (нам надо, чтобы дробь была периодической). Для этого "сжульничаем", заменив в определении переменной <code>hours</code> <code>cur_time()[0]</code> на <code>'11'</code>:

In [44]:
hours = (number_of_day(date()) - 1) * 24 + int('11')
title = str(9 + hours / 2400) + ' Д'
print(title)

9.114583333333334 Д


Это очень некрасиво. Давайте напишем код, который будет обрамлять период в красивенькие круглые скобки за адекватное время и адекватную память. Без доказательств воспользуемся следующим фактом: длина наименьшего предпериода правильной несократимой дроби со знаменателем $2^a5^bk$, где $a, \ b, \ k \in \mathbb{Z_+}$ и $\text{НОД}(k, 10) = 1$ -- это $max(a, b)$ (далее будем называть это Леммой 1).

Далее. Дробь нам, как видим, придётся сокращать, поэтому запасёмся алгоритмом Евклида:)

In [46]:
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

Теперь функция подсчёта длины предпериода с учётом Леммы 1:

In [48]:
def pre_period(n):
    f_counter = 0
    while n % 5 == 0:
        n //= 5
        f_counter += 1
    t_counter = 0
    while n % 2 == 0:
        n //= 2
        t_counter += 1
    return max(f_counter, t_counter)

А дальше -- простое деление в столбик: пока шагов меньше, чем длина предпериода, делим. Как только достигли конца предпериода, запоминаем остаток. И ждём повторения. Как только повторился -- всё, записываем период в круглые скобки. Реализуется двумя функциями:

In [50]:
def dec_period(a, b, PrePeriod):
    res = ""
    end = False
    counter = 0
    while not end:
        a %= b
        a *= 10
        if a == 0:
            end = True
        if counter == PrePeriod and not end:
            old_a = a
            res += "("
        elif counter > PrePeriod:
            if a == old_a:
                res += ")"
                end = True
        counter += 1
        if not end:
            res += str(a // b)
    return res

def periodic_number(a, b):
    a, b = a // gcd(a, b), b // gcd(a, b)
    ans = str(a // b) + "."
    a %= b
    PrePeriod = pre_period(b)
    return ans + dec_period(a, b, PrePeriod)

Ура! Теперь мы готовы написать окончательную функцию изменения названия беседы!

In [61]:
def change_name():
    global u_log, u_pas, uid
    hours = (number_of_day(date()) - 1) * 24 + int(cur_time()[0])
    vk = vk_api.VkApi(login = u_log, password = u_pas)
    vk.auth()
    vk.method('messages.editChat', {'chat_id':int(uid), 'title':periodic_number((9 * 2400 + hours), 2400)+ ' Д'})
    return 0

Тело программы совсем маленьким будет:

In [62]:
fin = open('fin.txt', 'r')
user_data = fin.readlines()[0].split()
u_log, u_pas, uid = user_data[0], user_data[1], user_data[2]
change_name()
print('Success')
#Дальше всё закомменчу, потому что если запустить, то работать будет до 1 сентября(
#time.sleep(to_next_hour(cur_time()) + 10)
#while date() != ['Sep', '1']:
    #change_name()
    #print('Success')
    #time.sleep(3600)
fin.close()

Success


ВКонтакте выдаёт чёт такое:
"Иван Карпов изменил название беседы: «9 Д» → «9.11541(6) Д»"

Далее я арендовал vps на ubuntu 16.04 и залил туда код c github (он лежит тут https://github.com/karp57/D20chat/blob/master/Date.py)
Если Вы так никогда не развлекались, то пошаговая инструкция (работаем чрез консольку):

Если python3 не устанволен заранее, то делаем <code>sudo apt install python3.6</code>
Далее, 
<br />
<code>sudo apt install python3-pip</code>
<br />
<code>sudo apt update</code>
<br />
<code>sudo apt-get install git</code>
<br />
<code>sudo apt update</code>
<br />
<code>pip3 install vk </code>
<br />
<code>pip3 install vk_api</code>
<br />
<code>git clone https://github.com/karp57/D20chat.git</code>
<br />
<code>cd ~/D20chat</code>
<br />
<code>touch fin.txt</code>
<br />
(Здесь нужно заполнить fin.txt по указанному где-то наверху этого текста шаблону через редактор Vim)
<br />
<code>python3 Date.py</code>

После этого оно должно заработать и выдать <code>Success</code>. Если не заработало -- пишите мне в https://vk.com/ivankarpov057 ! 